# **Import the Necessary Libraries**

In [80]:
import pandas as pd
import numpy as np
import os
import ast
from tqdm import tqdm

# **Read The Test Data**

In [2]:
train_path_urban = 'C:/Users/Vaideeswaran/Documents/ACS Project/Learning Dataset/test_urban_dataset.csv'
train_path_highway = 'C:/Users/Vaideeswaran/Documents/ACS Project/Learning Dataset/test_highway_dataset.csv'

df_urban = pd.read_csv(train_path_urban, usecols=["new_state"])
df_highway = pd.read_csv(train_path_highway, usecols=["new_state"])

# Convert the column to a list

new_state_list_urban = [ast.literal_eval(item) for item in tqdm(df_urban['new_state'])] # Convert string representation of lists to actual lists
new_state_list_highway = [ast.literal_eval(item) for item in tqdm(df_highway['new_state'])] # Convert string representation of lists to actual lists

# Find the length of the lists

length_urban = len(new_state_list_urban)
length_highway = len(new_state_list_highway)

print(f"Length of new_state_list_urban: {length_urban}")
print(f"Length of new_state_list_highway: {length_highway}")

100%|██████████| 8113281/8113281 [02:53<00:00, 46856.94it/s]

Length of new_state_list_urban: 7907169
Length of new_state_list_highway: 8113281


# **Finding The Optimal Threshold**

In [4]:
MCS_Table = {
    0:  ("QPSK", 120),
    1:  ("QPSK", 193),
    2:  ("QPSK", 308),
    3:  ("QPSK", 449),
    4:  ("QPSK", 602),
    5:  ("16QAM", 378),
    6:  ("16QAM", 434),
    7:  ("16QAM", 490),
    8:  ("16QAM", 553),
    9:  ("16QAM", 616),
    10: ("16QAM", 658),
    11: ("64QAM", 466),
    12: ("64QAM", 517),
    13: ("64QAM", 567),
    14: ("64QAM", 616),
    15: ("64QAM", 666),
    16: ("64QAM", 719),
    17: ("64QAM", 772),
    18: ("64QAM", 822),
    19: ("64QAM", 873),
    20: ("256QAM", 683),
    21: ("256QAM", 711),
    22: ("256QAM", 754),
    23: ("256QAM", 797),
    24: ("256QAM", 841),
    25: ("256QAM", 885),
    26: ("256QAM", 917),
    27: ("256QAM", 948)
}

In [5]:
vel_dict_urban_state = {}
vel_dict_urban_above_state = {}
v_max_urban = 120

for i in range(v_max_urban + 1):
    vel_dict_urban_state[i] = np.array([0, 0, 0])  # Initialize with a tuple (LOS, NLOS, NLOSv)
    vel_dict_urban_above_state[i] = np.array([0, 0, 0])  # Initialize with a tuple (LOS, NLOS, NLOSv)

# Extract velocity information from the new_state lists

for state in new_state_list_urban:
    vel = state[2]
    if int(vel) > v_max_urban:
        vel = v_max_urban

    if state[3] != state[4]:
        vel_dict_urban_state[int(vel)] = vel_dict_urban_state[int(vel)] + np.array([0, 1, 0])
    else:
        d = np.linalg.norm(np.array(state[0]) - np.array(state[1]))
        prob_LOS = min(1, 1.05 * np.exp(-0.0114 * d))

        if np.random.rand() < prob_LOS:
            vel_dict_urban_state[int(vel)] = vel_dict_urban_state[int(vel)] + np.array([1, 0, 0])
        else:
            vel_dict_urban_state[int(vel)] = vel_dict_urban_state[int(vel)] + np.array([0, 0, 1])

vel_dict_urban_above_state[59] = vel_dict_urban_state[59]
for i in range(58, -1, -1):
    vel_dict_urban_above_state[i] = vel_dict_urban_above_state[i + 1] + vel_dict_urban_state[i]

vel_dict_urban_above_state[119] = vel_dict_urban_state[119]
for i in range(118, 59, -1):
    vel_dict_urban_above_state[i] = vel_dict_urban_above_state[i + 1] + vel_dict_urban_state[i]

vel_dict_urban_above_state[120] = vel_dict_urban_state[120]

In [6]:
vel_dict_highway_state = {}
vel_dict_highway_above_state = {}
v_max_highway = 280

for i in range(v_max_highway + 1):
    vel_dict_highway_state[i] = np.array([0, 0])  # Initialize with a tuple (LOS, NLOSv)
    vel_dict_highway_above_state[i] = np.array([0, 0])  # Initialize with a tuple (LOS, NLOSv)

for state in new_state_list_highway:
    vel = state[2]
    if int(vel) > v_max_highway:
        vel = v_max_highway

    d = np.linalg.norm(np.array(state[0]) - np.array(state[1]))
    (a, b, c) = (2.1013e-6, -0.002, 1.0193)

    if d <= 475:
        prob_LOS = min(1, (a * d**2) + (b * d) + c)
    else:
        prob_LOS = max(0, 0.54 - 0.001 * (d - 475))

    if np.random.rand() < prob_LOS:
        vel_dict_highway_state[int(vel)] = vel_dict_highway_state[int(vel)] + np.array([1, 0])
    else:
        vel_dict_highway_state[int(vel)] = vel_dict_highway_state[int(vel)] + np.array([0, 1])

vel_dict_highway_above_state[69] = vel_dict_highway_state[69]
for i in range(68, -1, -1):
    vel_dict_highway_above_state[i] = vel_dict_highway_above_state[i + 1] + vel_dict_highway_state[i]

vel_dict_highway_above_state[139] = vel_dict_highway_state[139]
for i in range(138, 69, -1):
    vel_dict_highway_above_state[i] = vel_dict_highway_above_state[i + 1] + vel_dict_highway_state[i]

vel_dict_highway_above_state[279] = vel_dict_highway_state[279]
for i in range(278, 139, -1):
    vel_dict_highway_above_state[i] = vel_dict_highway_above_state[i + 1] + vel_dict_highway_state[i]

vel_dict_highway_above_state[280] = vel_dict_highway_state[280]

In [81]:
def BLER_TBS(base_path, SNR, scenario, channel_condition, speed_profile, MCS_LUT = MCS_Table):

    TBS_path = "C:/Users/Vaideeswaran/Documents/ACS Project/TBS Determination/output.csv"
    TBS_df = pd.read_csv(TBS_path)

    df = pd.read_csv(base_path)
    row = df[df['SNR'] == SNR]
    if row.empty:
        raise ValueError("No recommendation found for the given SNR.")
    
    recommendation = row.iloc[0]['Recommendation']

    if pd.isna(recommendation):
        return (1, 0, [0, 0, -1])
    subchannels, dmrs, mcs_index = ast.literal_eval(recommendation)
    modulation, rate = MCS_LUT[mcs_index]
    file_name = scenario + "_" + channel_condition + "_" + speed_profile + "_" + str(subchannels) + "Subch_" + str(dmrs) + "DMRS_" + modulation + "-" + str(rate) + ".csv"
    BLER_path = "C:/Users/Vaideeswaran/Documents/ACS Project/Monte Carlo Dataset/NRV2XSL_LinkLevelDatasetAugmented/" + scenario + "/" + channel_condition + "/" + speed_profile + "/" + str(subchannels) + "Subch/" + str(dmrs) + "DMRS/" + file_name
    bler_df = pd.read_csv(BLER_path)
    result = bler_df[bler_df['TB SNR [dB]'] == SNR]
    if not result.empty:
        BLER = float(result['TB BLER'].values[0])
    else:
        raise ValueError("No BLER found for the given SNR in the specified file.")
    
    match = TBS_df[
        (TBS_df['dmrsNum'] == dmrs) &
        (TBS_df['subchNum'] == subchannels) &
        (TBS_df['mcsIndex'] == mcs_index)
    ]

    if not match.empty:
        TBS = int(match['TBS'].values[0])
    else:
        raise ValueError("No TBS found for the given parameters in the specified file.")
    
    return (BLER, TBS, [dmrs, subchannels, mcs_index])

def threshold_identification(reliability, setup, SNR, vel_dict_urban_above_state = vel_dict_urban_above_state, vel_dict_highway_above_state = vel_dict_highway_above_state):

    if setup == "Urban":

        urban_lower_threshold_dict_LOS, urban_lower_threshold_dict_NLOS, urban_lower_threshold_dict_NLOSv = {}, {}, {}
        urban_lower_threshold_list = [i for i in range(0, 60, 1)]
        urban_higher_threshold_dict_LOS, urban_higher_threshold_dict_NLOS, urban_higher_threshold_dict_NLOSv = {}, {}, {}
        urban_higher_threshold_list = [i for i in range(60, 120, 1)]

        recommendation_base_path_1 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_NLOS_0kmph_"+ reliability + ".csv"
        (BLER_1, TBS_1, _) = BLER_TBS(recommendation_base_path_1, SNR, setup, "NLOS", "0kmph")

        recommendation_base_path_2 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_NLOS_60kmph_" + reliability + ".csv"
        (BLER_2, TBS_2, _) = BLER_TBS(recommendation_base_path_2, SNR, setup, "NLOS", "60kmph")
        
        recommendation_base_path_3 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_NLOS_120kmph_" + reliability + ".csv"
        (BLER_3, TBS_3, _) = BLER_TBS(recommendation_base_path_3, SNR, setup, "NLOS", "120kmph")

        recommendation_base_path_4 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_LOS_0kmph_" + reliability + ".csv"
        (BLER_4, TBS_4, _) = BLER_TBS(recommendation_base_path_4, SNR, setup, "LOS", "0kmph")
        
        recommendation_base_path_5 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_LOS_60kmph_" + reliability + ".csv"
        (BLER_5, TBS_5, _) = BLER_TBS(recommendation_base_path_5, SNR, setup, "LOS", "60kmph")

        recommendation_base_path_6 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_LOS_120kmph_" + reliability + ".csv"
        (BLER_6, TBS_6, _) = BLER_TBS(recommendation_base_path_6, SNR, setup, "LOS", "120kmph")

        recommendation_base_path_7 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_NLOSv_0kmph_" + reliability + ".csv"
        (BLER_7, TBS_7, _) = BLER_TBS(recommendation_base_path_7, SNR, setup, "NLOSv", "0kmph")

        recommendation_base_path_8 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_NLOSv_60kmph_" + reliability + ".csv"
        (BLER_8, TBS_8, _) = BLER_TBS(recommendation_base_path_8, SNR, setup, "NLOSv", "60kmph")

        recommendation_base_path_9 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Urban_NLOSv_120kmph_" + reliability + ".csv"
        (BLER_9, TBS_9, _) = BLER_TBS(recommendation_base_path_9, SNR, setup, "NLOSv", "120kmph")

        for i in urban_lower_threshold_list:

            factor1_LOS = vel_dict_urban_above_state[i][0]/vel_dict_urban_above_state[0][0]
            factor1_NLOS = vel_dict_urban_above_state[i][1]/vel_dict_urban_above_state[0][1]
            factor1_NLOSv = vel_dict_urban_above_state[i][2]/vel_dict_urban_above_state[0][2]

            factor2_LOS = (vel_dict_urban_above_state[0][0] - vel_dict_urban_above_state[i][0])/vel_dict_urban_above_state[0][0]
            factor2_NLOS = (vel_dict_urban_above_state[0][1] - vel_dict_urban_above_state[i][1])/vel_dict_urban_above_state[0][1]
            factor2_NLOSv = (vel_dict_urban_above_state[0][2] - vel_dict_urban_above_state[i][2])/vel_dict_urban_above_state[0][2]
            
            BLER_LOS = (BLER_5 * factor1_LOS) + (BLER_4 * factor2_LOS)
            BLER_NLOS = (BLER_2 * factor1_NLOS) + (BLER_1 * factor2_NLOS)
            BLER_NLOSv = (BLER_8 * factor1_NLOSv) + (BLER_7 * factor2_NLOSv)

            TBS_LOS = (TBS_5 * factor1_LOS) + (TBS_4 * factor2_LOS)
            TBS_NLOS = (TBS_2 * factor1_NLOS) + (TBS_1 * factor2_NLOS)
            TBS_NLOSv = (TBS_8 * factor1_NLOSv) + (TBS_7 * factor2_NLOSv)

            urban_lower_threshold_dict_LOS[i] = (BLER_LOS, TBS_LOS)
            urban_lower_threshold_dict_NLOS[i] = (BLER_NLOS, TBS_NLOS)
            urban_lower_threshold_dict_NLOSv[i] = (BLER_NLOSv, TBS_NLOSv)

        for i in urban_higher_threshold_list:

            factor3_LOS = vel_dict_urban_above_state[i][0]/vel_dict_urban_above_state[60][0]
            factor3_NLOS = vel_dict_urban_above_state[i][1]/vel_dict_urban_above_state[60][1]
            factor3_NLOSv = vel_dict_urban_above_state[i][2]/vel_dict_urban_above_state[60][2]

            factor4_LOS = (vel_dict_urban_above_state[60][0] - vel_dict_urban_above_state[i][0])/vel_dict_urban_above_state[60][0]
            factor4_NLOS = (vel_dict_urban_above_state[60][1] - vel_dict_urban_above_state[i][1])/vel_dict_urban_above_state[60][1]
            factor4_NLOSv = (vel_dict_urban_above_state[60][2] - vel_dict_urban_above_state[i][2])/vel_dict_urban_above_state[60][2]
            
            BLER_LOS = (BLER_6 * factor3_LOS) + (BLER_5 * factor4_LOS)
            BLER_NLOS = (BLER_3 * factor3_NLOS) + (BLER_2 * factor4_NLOS)
            BLER_NLOSv = (BLER_9 * factor3_NLOSv) + (BLER_8 * factor4_NLOSv)

            TBS_LOS = (TBS_6 * factor3_LOS) + (TBS_5 * factor4_LOS)
            TBS_NLOS = (TBS_3 * factor3_NLOS) + (TBS_2 * factor4_NLOS)
            TBS_NLOSv = (TBS_9 * factor3_NLOSv) + (TBS_8 * factor4_NLOSv)

            urban_higher_threshold_dict_LOS[i] = (BLER_LOS, TBS_LOS)
            urban_higher_threshold_dict_NLOS[i] = (BLER_NLOS, TBS_NLOS)
            urban_higher_threshold_dict_NLOSv[i] = (BLER_NLOSv, TBS_NLOSv)

        return urban_lower_threshold_dict_LOS, urban_lower_threshold_dict_NLOS, urban_lower_threshold_dict_NLOSv, urban_higher_threshold_dict_LOS, urban_higher_threshold_dict_NLOS, urban_higher_threshold_dict_NLOSv

    elif setup == "Highway":

        highway_lower_threshold_dict_LOS, highway_lower_threshold_dict_NLOSv = {}, {}
        highway_lower_threshold_list = [i for i in range(0, 70, 1)]
        highway_middle_threshold_dict_LOS, highway_middle_threshold_dict_NLOSv = {}, {}
        highway_middle_threshold_list = [i for i in range(70, 140, 1)]
        highway_higher_threshold_dict_LOS, highway_higher_threshold_dict_NLOSv = {}, {}
        highway_higher_threshold_list = [i for i in range(140, 280, 1)]

        recommendation_base_path_1 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_LOS_0kmph_" + reliability + ".csv"
        (BLER_1, TBS_1, _) = BLER_TBS(recommendation_base_path_1, SNR, setup, "LOS", "0kmph")

        recommendation_base_path_2 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_LOS_70kmph_" + reliability + ".csv"
        (BLER_2, TBS_2, _) = BLER_TBS(recommendation_base_path_2, SNR, setup, "LOS", "70kmph")

        recommendation_base_path_3 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_LOS_140kmph_" + reliability + ".csv"
        (BLER_3, TBS_3, _) = BLER_TBS(recommendation_base_path_3, SNR, setup, "LOS", "140kmph")

        recommendation_base_path_4 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_LOS_280kmph_" + reliability + ".csv"
        (BLER_4, TBS_4, _) = BLER_TBS(recommendation_base_path_4, SNR, setup, "LOS", "280kmph")

        recommendation_base_path_5 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_NLOSv_0kmph_" + reliability + ".csv"
        (BLER_5, TBS_5, comb5) = BLER_TBS(recommendation_base_path_5, SNR, setup, "NLOSv", "0kmph")

        recommendation_base_path_6 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_NLOSv_70kmph_" + reliability + ".csv"
        (BLER_6, TBS_6, comb6) = BLER_TBS(recommendation_base_path_6, SNR, setup, "NLOSv", "70kmph")

        recommendation_base_path_7 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_NLOSv_140kmph_" + reliability + ".csv"
        (BLER_7, TBS_7, _) = BLER_TBS(recommendation_base_path_7, SNR, setup, "NLOSv", "140kmph")

        recommendation_base_path_8 = "C:/Users/Vaideeswaran/Documents/ACS Project/Recommendations/recommendation_dict_Highway_NLOSv_280kmph_" + reliability + ".csv"
        (BLER_8, TBS_8, _) = BLER_TBS(recommendation_base_path_8, SNR, setup, "NLOSv", "280kmph")

        for i in highway_lower_threshold_list:

            factor1_LOS = vel_dict_highway_above_state[i][0]/vel_dict_highway_above_state[0][0]
            factor1_NLOSv = vel_dict_highway_above_state[i][1]/vel_dict_highway_above_state[0][1]

            factor2_LOS = (vel_dict_highway_above_state[0][0] - vel_dict_highway_above_state[i][0])/vel_dict_highway_above_state[0][0]
            factor2_NLOSv = (vel_dict_highway_above_state[0][1] - vel_dict_highway_above_state[i][1])/vel_dict_highway_above_state[0][1]

            BLER_LOS = (BLER_2 * factor1_LOS) + (BLER_1 * factor2_LOS)
            BLER_NLOSv = (BLER_6 * factor1_NLOSv) + (BLER_5 * factor2_NLOSv)

            TBS_LOS = (TBS_2 * factor1_LOS) + (TBS_1 * factor2_LOS)
            TBS_NLOSv = (TBS_6 * factor1_NLOSv) + (TBS_5 * factor2_NLOSv)

            highway_lower_threshold_dict_LOS[i] = (BLER_LOS, TBS_LOS)
            highway_lower_threshold_dict_NLOSv[i] = (BLER_NLOSv, TBS_NLOSv)

        for i in highway_middle_threshold_list:

            factor3_LOS = vel_dict_highway_above_state[i][0]/vel_dict_highway_above_state[70][0]
            factor3_NLOSv = vel_dict_highway_above_state[i][1]/vel_dict_highway_above_state[70][1]

            factor4_LOS = (vel_dict_highway_above_state[70][0] - vel_dict_highway_above_state[i][0])/vel_dict_highway_above_state[70][0]
            factor4_NLOSv = (vel_dict_highway_above_state[70][1] - vel_dict_highway_above_state[i][1])/vel_dict_highway_above_state[70][1]

            BLER_LOS = (BLER_3 * factor3_LOS) + (BLER_2 * factor4_LOS)
            BLER_NLOSv = (BLER_7 * factor3_NLOSv) + (BLER_6 * factor4_NLOSv)

            TBS_LOS = (TBS_3 * factor3_LOS) + (TBS_2 * factor4_LOS)
            TBS_NLOSv = (TBS_7 * factor3_NLOSv) + (TBS_6 * factor4_NLOSv)

            highway_middle_threshold_dict_LOS[i] = (BLER_LOS, TBS_LOS)
            highway_middle_threshold_dict_NLOSv[i] = (BLER_NLOSv, TBS_NLOSv)

        for i in highway_higher_threshold_list:

            factor5_LOS = vel_dict_highway_above_state[i][0]/vel_dict_highway_above_state[140][0]
            factor5_NLOSv = vel_dict_highway_above_state[i][1]/vel_dict_highway_above_state[140][1]

            factor6_LOS = (vel_dict_highway_above_state[140][0] - vel_dict_highway_above_state[i][0])/vel_dict_highway_above_state[140][0]
            factor6_NLOSv = (vel_dict_highway_above_state[140][1] - vel_dict_highway_above_state[i][1])/vel_dict_highway_above_state[140][1]

            BLER_LOS = (BLER_4 * factor5_LOS) + (BLER_3 * factor6_LOS)
            BLER_NLOSv = (BLER_8 * factor5_NLOSv) + (BLER_7 * factor6_NLOSv)

            TBS_LOS = (TBS_4 * factor5_LOS) + (TBS_3 * factor6_LOS)
            TBS_NLOSv = (TBS_8 * factor5_NLOSv) + (TBS_7 * factor6_NLOSv)

            highway_higher_threshold_dict_LOS[i] = (BLER_LOS, TBS_LOS)
            highway_higher_threshold_dict_NLOSv[i] = (BLER_NLOSv, TBS_NLOSv)

        return highway_lower_threshold_dict_LOS, highway_lower_threshold_dict_NLOSv, highway_middle_threshold_dict_LOS, highway_middle_threshold_dict_NLOSv, highway_higher_threshold_dict_LOS, highway_higher_threshold_dict_NLOSv

In [82]:
# Test the above function with the urban and highway datasets

# Urban case + High Reliability

urban_snr = [i for i in range(-13, 46, 1)]
urban_high_reliability_lower_threshold_list_LOS, urban_high_reliability_lower_threshold_list_NLOS, urban_high_reliability_lower_threshold_list_NLOSv = [], [], []
urban_high_reliability_higher_threshold_list_LOS, urban_high_reliability_higher_threshold_list_NLOS, urban_high_reliability_higher_threshold_list_NLOSv = [], [], []

for SNR in tqdm(urban_snr):
    # Call the function with the urban dataset
    urban_lower_thresholds_LOS, urban_lower_thresholds_NLOS, urban_lower_thresholds_NLOSv, urban_higher_thresholds_LOS, urban_higher_thresholds_NLOS, urban_higher_thresholds_NLOSv = threshold_identification("High", "Urban", SNR)
    urban_high_reliability_lower_threshold_list_LOS.append(urban_lower_thresholds_LOS)
    urban_high_reliability_lower_threshold_list_NLOS.append(urban_lower_thresholds_NLOS)
    urban_high_reliability_lower_threshold_list_NLOSv.append(urban_lower_thresholds_NLOSv)
    urban_high_reliability_higher_threshold_list_LOS.append(urban_higher_thresholds_LOS)
    urban_high_reliability_higher_threshold_list_NLOS.append(urban_higher_thresholds_NLOS)
    urban_high_reliability_higher_threshold_list_NLOSv.append(urban_higher_thresholds_NLOSv)

print("Done with Urban High Reliability Thresholds")

# Urban case + Low Reliability

urban_low_reliability_lower_thresholds_LOS, urban_low_reliability_lower_thresholds_NLOS, urban_low_reliability_lower_thresholds_NLOSv = [], [], []
urban_low_reliability_higher_thresholds_LOS, urban_low_reliability_higher_thresholds_NLOS, urban_low_reliability_higher_thresholds_NLOSv = [], [], []

for SNR in tqdm(urban_snr):
    # Call the function with the urban dataset
    urban_lower_thresholds_LOS, urban_lower_thresholds_NLOS, urban_lower_thresholds_NLOSv, urban_higher_thresholds_LOS, urban_higher_thresholds_NLOS, urban_higher_thresholds_NLOSv = threshold_identification("Low", "Urban", SNR)
    urban_low_reliability_lower_thresholds_LOS.append(urban_lower_thresholds_LOS)
    urban_low_reliability_lower_thresholds_NLOS.append(urban_lower_thresholds_NLOS)
    urban_low_reliability_lower_thresholds_NLOSv.append(urban_lower_thresholds_NLOSv)
    urban_low_reliability_higher_thresholds_LOS.append(urban_higher_thresholds_LOS)
    urban_low_reliability_higher_thresholds_NLOS.append(urban_higher_thresholds_NLOS)
    urban_low_reliability_higher_thresholds_NLOSv.append(urban_higher_thresholds_NLOSv)

print("Done with Urban Low Reliability Thresholds")

100%|██████████| 59/59 [00:01<00:00, 41.71it/s]


Done with Urban High Reliability Thresholds


100%|██████████| 59/59 [00:01<00:00, 42.16it/s]

Done with Urban Low Reliability Thresholds


In [83]:
# Highway case + High Reliability

highway_snr = [i for i in range(-13, 51, 1)]
highway_high_reliability_lower_threshold_list_LOS, highway_high_reliability_lower_threshold_list_NLOSv = [], []
highway_high_reliability_middle_threshold_list_LOS, highway_high_reliability_middle_threshold_list_NLOSv = [], []
highway_high_reliability_higher_threshold_list_LOS, highway_high_reliability_higher_threshold_list_NLOSv = [], []

for SNR in tqdm(highway_snr):
    # Call the function with the highway dataset
    highway_lower_thresholds_LOS, highway_lower_thresholds_NLOSv, highway_middle_thresholds_LOS, highway_middle_thresholds_NLOSv, highway_higher_thresholds_LOS, highway_higher_thresholds_NLOSv = threshold_identification("High", "Highway", SNR)
    highway_high_reliability_lower_threshold_list_LOS.append(highway_lower_thresholds_LOS)
    highway_high_reliability_lower_threshold_list_NLOSv.append(highway_lower_thresholds_NLOSv)
    highway_high_reliability_middle_threshold_list_LOS.append(highway_middle_thresholds_LOS)
    highway_high_reliability_middle_threshold_list_NLOSv.append(highway_middle_thresholds_NLOSv)
    highway_high_reliability_higher_threshold_list_LOS.append(highway_higher_thresholds_LOS)
    highway_high_reliability_higher_threshold_list_NLOSv.append(highway_higher_thresholds_NLOSv)

print("Done with Highway High Reliability Thresholds")

# Highway case + Low Reliability

highway_low_reliability_lower_threshold_list_LOS, highway_low_reliability_lower_threshold_list_NLOSv= [], []
highway_low_reliability_middle_threshold_list_LOS, highway_low_reliability_middle_threshold_list_NLOSv = [], []
highway_low_reliability_higher_threshold_list_LOS, highway_low_reliability_higher_threshold_list_NLOSv = [], []

for SNR in tqdm(highway_snr):
    # Call the function with the highway dataset
    highway_lower_thresholds_LOS, highway_lower_thresholds_NLOSv, highway_middle_thresholds_LOS, highway_middle_thresholds_NLOSv, highway_higher_thresholds_LOS, highway_higher_thresholds_NLOSv = threshold_identification("Low", "Highway", SNR)
    highway_low_reliability_lower_threshold_list_LOS.append(highway_lower_thresholds_LOS)
    highway_low_reliability_lower_threshold_list_NLOSv.append(highway_lower_thresholds_NLOSv)
    highway_low_reliability_middle_threshold_list_LOS.append(highway_middle_thresholds_LOS)
    highway_low_reliability_middle_threshold_list_NLOSv.append(highway_middle_thresholds_NLOSv)
    highway_low_reliability_higher_threshold_list_LOS.append(highway_higher_thresholds_LOS)
    highway_low_reliability_higher_threshold_list_NLOSv.append(highway_higher_thresholds_NLOSv)

print("Done with Highway Low Reliability Thresholds")

100%|██████████| 64/64 [00:01<00:00, 48.10it/s]


Done with Highway High Reliability Thresholds


100%|██████████| 64/64 [00:01<00:00, 47.79it/s]

Done with Highway Low Reliability Thresholds


In [84]:
# Save the results to CSV files - Urban

urban_high_reliability_lower_thresholds_df_LOS = pd.DataFrame(urban_high_reliability_lower_threshold_list_LOS)
urban_high_reliability_lower_thresholds_df_NLOS = pd.DataFrame(urban_high_reliability_lower_threshold_list_NLOS)
urban_high_reliability_lower_thresholds_df_NLOSv = pd.DataFrame(urban_high_reliability_lower_threshold_list_NLOSv)
urban_high_reliability_higher_thresholds_df_LOS = pd.DataFrame(urban_high_reliability_higher_threshold_list_LOS)
urban_high_reliability_higher_thresholds_df_NLOS = pd.DataFrame(urban_high_reliability_higher_threshold_list_NLOS)
urban_high_reliability_higher_thresholds_df_NLOSv = pd.DataFrame(urban_high_reliability_higher_threshold_list_NLOSv)
urban_high_reliability_lower_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_high_reliability_lower_thresholds_LOS.csv", index=False)
urban_high_reliability_lower_thresholds_df_NLOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_high_reliability_lower_thresholds_NLOS.csv", index=False)
urban_high_reliability_lower_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_high_reliability_lower_thresholds_NLOSv.csv", index=False)
urban_high_reliability_higher_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_high_reliability_higher_thresholds_LOS.csv", index=False)
urban_high_reliability_higher_thresholds_df_NLOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_high_reliability_higher_thresholds_NLOS.csv", index=False)
urban_high_reliability_higher_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_high_reliability_higher_thresholds_NLOSv.csv", index=False)

####################################################################################################################################################################################

urban_low_reliability_lower_thresholds_df_LOS = pd.DataFrame(urban_low_reliability_lower_thresholds_LOS)
urban_low_reliability_lower_thresholds_df_NLOS = pd.DataFrame(urban_low_reliability_lower_thresholds_NLOS)
urban_low_reliability_lower_thresholds_df_NLOSv = pd.DataFrame(urban_low_reliability_lower_thresholds_NLOSv)
urban_low_reliability_higher_thresholds_df_LOS = pd.DataFrame(urban_low_reliability_higher_thresholds_LOS)
urban_low_reliability_higher_thresholds_df_NLOS = pd.DataFrame(urban_low_reliability_higher_thresholds_NLOS)
urban_low_reliability_higher_thresholds_df_NLOSv = pd.DataFrame(urban_low_reliability_higher_thresholds_NLOSv)
urban_low_reliability_lower_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_low_reliability_lower_thresholds_LOS.csv", index=False)
urban_low_reliability_lower_thresholds_df_NLOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_low_reliability_lower_thresholds_NLOS.csv", index=False)
urban_low_reliability_lower_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_low_reliability_lower_thresholds_NLOSv.csv", index=False)
urban_low_reliability_higher_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_low_reliability_higher_thresholds_LOS.csv", index=False)
urban_low_reliability_higher_thresholds_df_NLOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_low_reliability_higher_thresholds_NLOS.csv", index=False)
urban_low_reliability_higher_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban/urban_low_reliability_higher_thresholds_NLOSv.csv", index=False)

####################################################################################################################################################################################

# Save the results to CSV files - Highway

highway_high_reliability_lower_thresholds_df_LOS = pd.DataFrame(highway_high_reliability_lower_threshold_list_LOS)
highway_high_reliability_lower_thresholds_df_NLOSv = pd.DataFrame(highway_high_reliability_lower_threshold_list_NLOSv)
highway_high_reliability_middle_thresholds_df_LOS = pd.DataFrame(highway_high_reliability_middle_threshold_list_LOS)
highway_high_reliability_middle_thresholds_df_NLOSv = pd.DataFrame(highway_high_reliability_middle_threshold_list_NLOSv)
highway_high_reliability_higher_thresholds_df_LOS = pd.DataFrame(highway_high_reliability_higher_threshold_list_LOS)
highway_high_reliability_higher_thresholds_df_NLOSv = pd.DataFrame(highway_high_reliability_higher_threshold_list_NLOSv)
highway_high_reliability_lower_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_high_reliability_lower_thresholds_LOS.csv", index=False)
highway_high_reliability_lower_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_high_reliability_lower_thresholds_NLOSv.csv", index=False)
highway_high_reliability_middle_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_high_reliability_middle_thresholds_LOS.csv", index=False)
highway_high_reliability_middle_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_high_reliability_middle_thresholds_NLOSv.csv", index=False)
highway_high_reliability_higher_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_high_reliability_higher_thresholds_LOS.csv", index=False)
highway_high_reliability_higher_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_high_reliability_higher_thresholds_NLOSv.csv", index=False)

####################################################################################################################################################################################

highway_low_reliability_lower_thresholds_df_LOS = pd.DataFrame(highway_low_reliability_lower_threshold_list_LOS)
highway_low_reliability_lower_thresholds_df_NLOSv = pd.DataFrame(highway_low_reliability_lower_threshold_list_NLOSv)
highway_low_reliability_middle_thresholds_df_LOS = pd.DataFrame(highway_low_reliability_middle_threshold_list_LOS)
highway_low_reliability_middle_thresholds_df_NLOSv = pd.DataFrame(highway_low_reliability_middle_threshold_list_NLOSv)
highway_low_reliability_higher_thresholds_df_LOS = pd.DataFrame(highway_low_reliability_higher_threshold_list_LOS)
highway_low_reliability_higher_thresholds_df_NLOSv = pd.DataFrame(highway_low_reliability_higher_threshold_list_NLOSv)
highway_low_reliability_lower_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_low_reliability_lower_thresholds_LOS.csv", index=False)
highway_low_reliability_lower_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_low_reliability_lower_thresholds_NLOSv.csv", index=False)
highway_low_reliability_middle_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_low_reliability_middle_thresholds_LOS.csv", index=False)
highway_low_reliability_middle_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_low_reliability_middle_thresholds_NLOSv.csv", index=False)
highway_low_reliability_higher_thresholds_df_LOS.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_low_reliability_higher_thresholds_LOS.csv", index=False)
highway_low_reliability_higher_thresholds_df_NLOSv.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway/highway_low_reliability_higher_thresholds_NLOSv.csv", index=False)

# **Threshold Determination**

In [97]:
high_reliability_threshold = 0.01
low_reliability_threshold = 0.1

def process_file(file_path, bler_threshold, snr_range):
    df = pd.read_csv(file_path)
    best_speeds = {}

    for idx, row in df.iterrows():
        best_speed = "NA"
        max_tbs = -1

        for col in df.columns:
            bler, tbs = ast.literal_eval(str(row[col]))
            if bler <= bler_threshold and tbs > max_tbs:
                max_tbs = tbs
                best_speed = col

        # Use actual SNR instead of row index
        snr = snr_range[idx]  
        best_speeds[snr] = best_speed

    return best_speeds


# Set your directory and BLER threshold
dir_urban = "C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Urban"
urban_SNR_range = [i for i in range(-13, 46, 1)]  # Example SNR range for urban

# Process all CSVs in the directory
result_dict_urban = {}
for filename in os.listdir(dir_urban):
    if filename.endswith(".csv"):
        file_path = os.path.join(dir_urban, filename)

        if "low_reliability" in filename:
            bler_threshold = low_reliability_threshold
        else:
            bler_threshold = high_reliability_threshold
        
        best_speeds = process_file(file_path, bler_threshold, urban_SNR_range)
        result_dict_urban[filename] = best_speeds

# Save the results to a new CSV file
output_df_urban = pd.DataFrame(result_dict_urban)
output_df_urban.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/best_speeds_urban.csv", index=True)

# Process the highway directory similarly
dir_highway = "C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/Highway"
highway_SNR_range = [i for i in range(-13, 51, 1)]  # Example SNR range for highway

# Process all CSVs in the highway directory
result_dict_highway = {}
for filename in os.listdir(dir_highway):
    if filename.endswith(".csv"):
        file_path = os.path.join(dir_highway, filename)

        if "low_reliability" in filename:
            bler_threshold = low_reliability_threshold
        else:
            bler_threshold = high_reliability_threshold

        best_speeds = process_file(file_path, bler_threshold, highway_SNR_range)
        result_dict_highway[filename] = best_speeds

# Save the results to a new CSV file for highway
output_df_highway = pd.DataFrame(result_dict_highway)
output_df_highway.to_csv("C:/Users/Vaideeswaran/Documents/ACS Project/Threshold Determination/best_speeds_highway.csv", index=True)